# Model Evaluation

Comparing chatbot's results based on different parameters tested

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read the UAT data into a DataFrame named uat_df
uat_df = pd.read_excel('UAT.xlsx')
uat_df.sample(5)


,Question,Bloom's Taxonomy,Question.1,Expected answer,source doc,source chunck,MMR,Similarity threshold,k value,Jame's ans,cosine-similarity matrix,Best way?,Best k
13,3,Apply,What are some cases in which WICA can be clai...,Employee met with a traffic accident while tak...,Work Injury Compensation Act (by MOM) - Comple...,1-10. WICA 2019 AT A GLANCE Who is covered? Al...,NaN,0.8,10,"Under WICA, employees can claim for various ca...",NaN,NaN,NaN
7,2,Understand,How to explain PCP in a simple way for migrant...,The PCP aims to provide eligible workers with ...,Primary Care Plan (by MOM).pdf,1-5. What is the Primary Care Plan (PCP) You m...,NaN,0.8,5,PCP stands for Primary Care Partnership. It is...,NaN,NaN,NaN
17,4,Analyze,What is the difference between WICA and PCP,PCP is a plan that provides eligible workers w...,Primary Care Plan (by MOM).pdf,1-5. What is the Primary Care Plan (PCP) You m...,NaN,0.8,5,WICA stands for the Work Injury Compensation A...,NaN,NaN,NaN
3,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-10. What is the Primary Care Plan (PCP) You ...,NaN,0.8,10,The Personal Care Assistant (PCA) program is d...,NaN,NaN,NaN
6,2,Understand,How to explain PCP in a simple way for migrant...,The PCP aims to provide eligible workers with ...,Primary Care Plan (by MOM).pdf,1-8. What is the Primary Care Plan (PCP) You m...,NaN,0.5,10,The Primary Care Plan (PCP) is a healthcare pl...,NaN,NaN,NaN


In [3]:
# change NaN to 0 in mmr column
uat_df['MMR'] = uat_df['MMR'].fillna(0)

# drop last 3 columns
uat_df = uat_df.drop(uat_df.columns[[10, 11, 12]], axis=1)

uat_df.head()

,Question,Bloom's Taxonomy,Question.1,Expected answer,source doc,source chunck,MMR,Similarity threshold,k value,Jame's ans
0,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-3 . What is the Primary Care Plan (PCP) You ...,0.0,0.5,5,The Primary Care Plan (PCP) is mandatory for W...
1,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-7. What is the Primary Care Plan (PCP) You m...,0.0,0.5,10,The Primary Care Plan (PCP) is mandatory for W...
2,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-5. What is the Primary Care Plan (PCP) You m...,0.0,0.8,5,The Personal Care Assistant (PCA) program is d...
3,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-10. What is the Primary Care Plan (PCP) You ...,0.0,0.8,10,The Personal Care Assistant (PCA) program is d...
4,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-5. What is the Primary Care Plan (PCP) You m...,1.0,NaN,5,The Primary Care Plan (PCP) is mandatory for W...


In [4]:
# for everyd data row, calculate cosine similarity between expected answer & james answer

# Import libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define a function that takes in two strings and calculates their cosine similarity
def calculate_cosine_similarity(string1, string2):
    # Create a CountVectorizer object
    count_vectorizer = CountVectorizer()
    # Transform the input strings using count_vectorizer
    count_vectorizer.fit([string1, string2])
    string1_vector = count_vectorizer.transform([string1])
    string2_vector = count_vectorizer.transform([string2])
    # Calculate the cosine similarity between the two vectors
    cosine = cosine_similarity(string1_vector, string2_vector)[0][0]
    return cosine


In [5]:

# Define a function that takes in a row and calculates the cosine similarity between the expected answer and James's answer
# add the cosine value to the row under a column named Cosine Similarity
for index, row in uat_df.iterrows():
    uat_df.loc[index, 'Cosine Similarity'] = calculate_cosine_similarity(row['Expected answer'], row["Jame's ans"])

In [6]:
# Read output data into a excel
uat_df.to_excel('output.xlsx', index=False)

In [7]:
# Calculate average mean of cosine similarity between MMR and Similarity Search
uat_df.groupby('MMR')['Cosine Similarity'].mean()

MMR
0.0    0.518847
1.0    0.564454
Name: Cosine Similarity, dtype: float64

In [10]:
# From MMR, group by k and calculate average mean of cosine similarity

uat_df.groupby(['MMR', 'k value'])['Cosine Similarity'].mean()

MMR  k value
0.0  5          0.517977
     10         0.519716
1.0  5          0.564454
Name: Cosine Similarity, dtype: float64